### Importing Libraries

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from queue import PriorityQueue
import sys

In [2]:
def heuristic(current, goal):
    """Manhattan distance heuristic for A* algorithm."""
    return abs(current[0] - goal[0]) + abs(current[1] - goal[1])


In [3]:
def reconstruct_path(came_from, start, end):
    """Reconstruct the path from start to end based on the came_from dictionary."""
    path = [end]
    while path[-1] != start:
        path.append(came_from[tuple(path[-1])])
    path.reverse()
    return path


In [4]:
def get_neighbors(maze, cell):
    """Get the neighboring cells of the given cell in the maze."""
    row, col = cell  # Unpack row and col from cell
    neighbors = []

    # Check the cell above
    if row > 0 and not maze[row - 1][col]:
        neighbors.append((row - 1, col))

    # Check the cell below
    if row < len(maze) - 1 and not maze[row + 1][col]:
        neighbors.append((row + 1, col))

    # Check the cell to the left
    if col > 0 and not maze[row][col - 1]:
        neighbors.append((row, col - 1))

    # Check the cell to the right
    if col < len(maze[0]) - 1 and not maze[row][col + 1]:
        neighbors.append((row, col + 1))

    return neighbors


In [5]:
def a_star(maze, start, end):
    """Find a path from start to end in the given maze using A* algorithm."""
    open_list = [(start, 0)]
    came_from = {}
    g_score = {start: 0}
    f_score = {start: heuristic(start, end)}

    while open_list:
        current, _ = min(open_list, key=lambda x: x[1])
        if current == end:
            return True, reconstruct_path(came_from, start, end)

        new_open_list = []
        for item in open_list:
            if item[0] != current:
                new_open_list.append(item)
        open_list = new_open_list

        for neighbor in get_neighbors(maze, current):
            tentative_g_score = g_score[current] + 1  # Assuming all edges have a cost of 1
            if tentative_g_score < g_score.get(neighbor, float('inf')):
                came_from[neighbor] = current
                g_score[neighbor] = tentative_g_score
                f_score[neighbor] = tentative_g_score + heuristic(neighbor, end)
                if neighbor not in [item[0] for item in open_list]:
                    open_list.append((neighbor, f_score[neighbor]))

    # If open_list is empty and we haven't reached the end, then there is no path
    return False, []


In [6]:
import pygame
import numpy as np

# Define colors
COLOR_WHITE = (255, 255, 255)
COLOR_BLACK = (0, 0, 0)
COLOR_GREEN = (0, 255, 0)
COLOR_RED = (255, 0, 0)
COLOR_BLUE = (0, 0, 255)

# Define cell size and gap between cells
CELL_SIZE = 20
CELL_GAP = 2

# Define maze size
n = 10

# Initialize pygame
pygame.init()

# Set up display
display_width = n * CELL_SIZE + (n + 1) * CELL_GAP
display_height = n * CELL_SIZE + (n + 1) * CELL_GAP
display = pygame.display.set_mode((display_width, display_height))
pygame.display.set_caption('Maze Visual Input')

# Create maze and visualization grid
maze = np.zeros((n, n), dtype=int)
visualization_grid = np.zeros((n, n), dtype=int)

# Set start and end positions
start_pos = (0, 0)
end_pos = (n - 1, n - 1)
visualization_grid[start_pos] = 2
visualization_grid[end_pos] = 3

# Main loop for visual input
while True:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            pygame.quit()
            sys.exit()
        elif event.type == pygame.MOUSEBUTTONDOWN:
            # Get cell coordinates from mouse click
            x, y = event.pos
            row = (y - CELL_GAP) // (CELL_SIZE + CELL_GAP)
            col = (x - CELL_GAP) // (CELL_SIZE + CELL_GAP)

            if 0 <= row < n and 0 <= col < n:
                if visualization_grid[row, col] == 0:
                    visualization_grid[row, col] = 1
                    maze[row, col] = 1
                elif visualization_grid[row, col] == 1:
                    visualization_grid[row, col] = 0
                    maze[row, col] = 0
                elif visualization_grid[row, col] == 2:
                    visualization_grid[start_pos] = 0
                    start_pos = (row, col)
                    visualization_grid[start_pos] = 2
                elif visualization_grid[row, col] == 3:
                    visualization_grid[end_pos] = 0
                    end_pos = (row, col)
                    visualization_grid[end_pos] = 3
        
        elif event.type == pygame.KEYDOWN and event.key == pygame.K_RETURN:
            # Run A* algorithm
            found, path = a_star(maze, start_pos, end_pos)

            if found:
                # Update visualization grid with path
                for pos in path:
                    if visualization_grid[pos] == 0:
                        visualization_grid[pos] = 4

                print("Path found!")
            else:
                print("No path found.")
        
    # Draw maze grid
    display.fill(COLOR_WHITE)
    for row in range(n):
        for col in range(n):
            x = col * (CELL_SIZE + CELL_GAP) + CELL_GAP
            y = row * (CELL_SIZE + CELL_GAP) + CELL_GAP
            if visualization_grid[row, col] == 0:
                pygame.draw.rect(display, COLOR_WHITE, pygame.Rect(x, y, CELL_SIZE, CELL_SIZE))
            elif visualization_grid[row, col] == 1:
                pygame.draw.rect(display, COLOR_BLACK, pygame.Rect(x, y, CELL_SIZE, CELL_SIZE))
            elif visualization_grid[row, col] == 2:
                pygame.draw.rect(display, COLOR_GREEN, pygame.Rect(x, y, CELL_SIZE, CELL_SIZE))
            elif visualization_grid[row, col] == 3:
                pygame.draw.rect(display, COLOR_RED, pygame.Rect(x, y, CELL_SIZE, CELL_SIZE))
            elif visualization_grid[row, col] == 4:
                pygame.draw.rect(display, COLOR_BLUE, pygame.Rect(x, y, CELL_SIZE, CELL_SIZE))
    pygame.display.flip()


pygame 2.2.0 (SDL 2.0.22, Python 3.9.13)
Hello from the pygame community. https://www.pygame.org/contribute.html
No path found.


KeyboardInterrupt: 